[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/PetiteIA/schema_mechanism/blob/master/notebooks/agent11.ipynb)

# L'AGENT QUI PRENAIT SON TEMPS (EN CONSTRUCTION)

# Objectifs

Ce tutoriel montre comment implémenter un agent qui apprend récursivement et réutilise à bon escient des séquences hiérarchiques de plus en plus haut niveau.

In [ ]:
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install ipywidgets
!pip install IPython.display

# Préparons les classes CompositeInteraction et Interaction

Nous conservons la même classe `CompositeInteraction` que l'Agent10.

In [7553]:
class CompositeInteraction:
    """A composite interaction is a tuple (pre_interaction, post_interaction) and a weight"""
    def __init__(self, pre_interaction, post_interaction):
        self.pre_interaction = pre_interaction
        self.post_interaction = post_interaction
        self.weight = 1
        self._step = 1

    def get_decision(self):
        """Return the flatten sequence of intermediary primitive interactions terminated with the final decision"""
        return f"{self.pre_interaction.sequence()}{self.post_interaction.get_decision()}"

    def get_actions(self):
        """Return the flat sequence of the decisions of this interaction as a string"""
        return f"{self.pre_interaction.get_actions()}{self.post_interaction.get_actions()}"
    
    def get_primitive_action(self):
        """Return the primite action"""
        return self.pre_interaction.get_primitive_action()

    def get_valence(self):
        """Return the valence of the pre_interaction plus the valence of the post_interaction"""
        return self.pre_interaction.get_valence() + self.post_interaction.get_valence()

    def reinforce(self):
        """Increment the composite interaction's weight"""
        self.weight += 1

    def key(self):
        """ The key to find this interaction in the dictionary is the string '<pre_interaction><post_interaction>'. """
        return f"({self.pre_interaction.key()},{self.post_interaction.key()})"

    def pre_key(self):
        """Return the key of the pre_interaction"""
        return self.pre_interaction.pre_key()

    def __str__(self):
        """ Print the interaction in the Newick tree format (pre_interaction, post_interaction: valence) """
        return f"({self.pre_interaction}, {self.post_interaction}: {self.weight})"

    def __eq__(self, other):
        """ Interactions are equal if they have the same pre and post interactions """
        if isinstance(other, self.__class__):
            return (self.pre_interaction == other.pre_interaction) and (self.post_interaction == other.post_interaction)
        else:
            return False

    def get_length(self):
        """Return the length of the number of primitive interactions in this composite interaction"""
        return self.pre_interaction.get_length() + self.post_interaction.get_length()

    def increment(self, interaction, interactions):
        """Increment the step of the appropriate sub-interaction. Return the enacted interaction if it is over, or None if it is ongoing."""
        # First step 
        if self._step == 1:
            interaction = self.pre_interaction.increment(interaction, interactions)
            # Ongoing pre-interaction. Return None
            if interaction is None:
                return None
            # Pre-interaction succeeded. Increment the step and return None
            elif interaction == self.pre_interaction:
                self._step = 2
                return None
            # Pre-interaction failed. Reset the step and return the enacted interaction
            else:
                self._step = 1
                return interaction
        # Second step
        else:
            interaction = self.post_interaction.increment(interaction, interactions)
            # Ongoing post-interaction. Return None
            if interaction is None:
                return None
            # Post-interaction succeeded. Reset the step and return this interaction
            elif interaction == self.post_interaction:
                self._step = 1
                return self
            # Post-interaction failed. Reset the step and return the enacted interaction
            else:
                self._step = 1
                composite_interaction = CompositeInteraction(self.pre_interaction, interaction)
                if composite_interaction.key() not in interactions:
                    # Add the enacted composite interaction to memory
                    interactions[composite_interaction.key()] = composite_interaction
                    print(f"Learning {composite_interaction}")
                    return composite_interaction
                else:
                    # Reinforce the existing composite interaction and return it
                    interactions[composite_interaction.key()].reinforce()
                    print(f"Reinforcing {interactions[composite_interaction.key()]}")
                    return interactions[composite_interaction.key()]

    def current(self):
        """Return the current intended primitive interaction"""
        # Step 1: the current primitive interaction of the pre-interaction
        if self._step == 1:
            return self.pre_interaction.current()
        # Step 2: The current primitive interaction of the post-interaction
        else:
            return self.post_interaction.current()

    def sequence(self):
        """Return the flat sequence of primitive interactions of this composite interaction"""
        return f"{self.pre_interaction.sequence()}{self.post_interaction.sequence()}"


Nous conservons la mêmes classe `Interaction ` que pour l'Agent10 sauf que nous ajoutons l'attribut `weight` initalisé à 10. 

In [7571]:
class Interaction:
    """An interaction is a tuple (action, outcome) with a valence"""
    def __init__(self, _action, _outcome, _valence):
        self._action = _action
        self._outcome = _outcome
        self._valence = _valence
        self.weight = 10
        
    def get_action(self):
        """Return the action"""
        return self._action

    def get_actions(self):
        """Return the action as a string for compatibilty with CompositeInteraction"""
        return str(self._action)

    def get_decision(self):
        """Return the decision key"""
        return f"{self._action}"
        # return f"a{self._action}"

    def get_primitive_action(self):
        """Return the action for compatibility with CompositeInteraction"""
        return self._action

    def get_outcome(self):
        """Return the action"""
        return self._outcome

    def get_valence(self):
        """Return the action"""
        return self._valence

    def key(self):
        """ The key to find this interaction in the dictinary is the string '<action><outcome>'. """
        return f"{self._action}{self._outcome}"

    def pre_key(self):
        """Return the key. Used for compatibility with CompositeInteraction"""
        return self.key()

    def __str__(self):
        """ Print interaction in the form '<action><outcome:<valence>' for debug."""
        return f"{self._action}{self._outcome}:{self._valence}"

    def __eq__(self, other):
        """ Interactions are equal if they have the same key """
        if isinstance(other, self.__class__):
            return self.key() == other.key()
        else:
            return False

    def get_length(self):
        """The length of the sequence of this interaction"""
        return 1

    def increment(self, interaction, interactions):
        """Return the enacted interaction for compatibility with composite interactions"""
        return interaction

    def current(self):
        """Return itself for compatibility with composite interactions"""
        return self

    def sequence(self):
        """Return the key. Use for compatibility with composite interactions"""
        return self.key()

# Implémentons l'agent 

In [7572]:
import pandas as pd

class Agent:
    def __init__(self, _interactions):
        """ Initialize our agent """
        self._interactions = {interaction.key(): interaction for interaction in _interactions}
        self._primitive_intended_interaction = self._interactions["00"]
        self._intended_interaction = None

        # The context
        self._penultimate_interaction = None
        self._previous_interaction = None
        self._last_interaction = None
        self._penultimate_composite_interaction = None
        self._previous_composite_interaction = None
        self._last_composite_interaction = None
        
        # Prepare the dataframe of proposed interactions
        default_interactions = [interaction for interaction in _interactions if interaction.get_outcome() == 0]
        data = {'activated': [""] * len(default_interactions),
                'weight': [0] * len(default_interactions),
                # 'action': [i.get_action() for i in default_interactions],
                'actions': [i.get_actions() for i in default_interactions],
                'intention': [i.key() for i in default_interactions],
                'valence': [i.get_valence() for i in default_interactions],
                'decision': [i.get_decision() for i in default_interactions],
                'length': [1] * len(default_interactions),
                'primitive': [i.key() for i in default_interactions]}
        self._default_df = pd.DataFrame(data)
        self.proposed_df = None
        self.decision_df = None
        self.clear = True # Used to clear the display after the enacted interaction

    def action(self, _outcome):
        """Implement the agent's policy"""

        # Trace the previous cycle
        primitive_enacted_interaction = self._interactions[f"{self._primitive_intended_interaction.get_action()}{_outcome}"]
        print(
            f"Action: {self._primitive_intended_interaction.get_action()}, Prediction: {self._primitive_intended_interaction.get_outcome()}, "
            f"Outcome: {_outcome}, Prediction_correct: {self._primitive_intended_interaction.get_outcome() == _outcome}, "
            f"Valence: {primitive_enacted_interaction.get_valence()}")

        # Follow up the enaction
        if self._intended_interaction is None: # First interaction cycle
            enacted_interaction = primitive_enacted_interaction
        else:
            enacted_interaction = self._intended_interaction.increment(primitive_enacted_interaction, self._interactions)

        # If the intended interaction is over (completely enacted or aborted)
        if enacted_interaction is None:
            self.clear = False
        else:
            self.clear = True
            # Memorize the context
            self._penultimate_composite_interaction = self._previous_composite_interaction
            self._previous_composite_interaction = self._last_composite_interaction
            self._penultimate_interaction = self._previous_interaction
            self._previous_interaction = self._last_interaction
            self._last_interaction = enacted_interaction
            # Call the learning mechanism
            self.learn(enacted_interaction)
            # Create the proposed dataframe
            self.create_proposed_df()
            self.aggregate_propositions()
            # Decide the next enaction
            self.decide()

        # Return the next primitive action
        self._primitive_intended_interaction = self._intended_interaction.current()
        return self._primitive_intended_interaction.get_action()
        
    def learn(self, enacted_interaction):
        """Learn the composite interactions"""
        # First level of composite interactions
        self._last_composite_interaction = self.learn_composite_interaction(self._previous_interaction, enacted_interaction)
        # Second level of composite interactions
        self.learn_composite_interaction(self._previous_composite_interaction, enacted_interaction)
        self.learn_composite_interaction(self._penultimate_interaction, self._last_composite_interaction)

        # Higher level composite interaction made of two composite interactions
        if self._last_composite_interaction is not None:
            self.learn_composite_interaction(self._penultimate_composite_interaction, self._last_composite_interaction)

    def learn_composite_interaction(self, pre_interaction, post_interaction):
        """Record or reinforce the composite interaction made of (pre_interaction, post_interaction)"""
        if pre_interaction is None:
            return None
        else:
            # If the pre-interaction exists
            composite_interaction = CompositeInteraction(pre_interaction, post_interaction)
            if composite_interaction.key() not in self._interactions:
                # Add the composite interaction to memory
                self._interactions[composite_interaction.key()] = composite_interaction
                print(f"Learning {composite_interaction}")
                return composite_interaction
            else:
                # Reinforce the existing composite interaction and return it
                self._interactions[composite_interaction.key()].reinforce()
                print(f"Reinforcing {self._interactions[composite_interaction.key()]}")
                return self._interactions[composite_interaction.key()]

    def create_proposed_df(self):
        """Create the proposed dataframe from the activated interactions"""
        # The list of activated interaction that match the current context
        activated_keys = [composite_interaction.key() for composite_interaction in self._interactions.values()
                          if composite_interaction.get_length() > 1 
                          and (composite_interaction.pre_interaction == self._last_interaction 
                               or composite_interaction.pre_interaction == self._last_composite_interaction
                               or (self._last_interaction.get_length() > 1
                                  and composite_interaction.pre_interaction == self._last_interaction.post_interaction))]
        data = {'activated': activated_keys,
                'weight': [self._interactions[k].weight for k in activated_keys],
                # 'action': [self._interactions[k].post_interaction.get_primitive_action() for k in activated_keys],
                'actions': [self._interactions[k].post_interaction.get_actions() for k in activated_keys],
                'intention': [self._interactions[k].post_interaction.key() for k in activated_keys],
                'valence': [self._interactions[k].post_interaction.get_valence() for k in activated_keys],
                'decision': [self._interactions[k].post_interaction.get_decision() for k in activated_keys],
                'primitive': [self._interactions[k].post_interaction.pre_key() for k in activated_keys],
                'length': [self._interactions[k].post_interaction.get_length() for k in activated_keys],
                }
        activated_df = pd.DataFrame(data).astype(self._default_df.dtypes)  # Force the same types for the case it is empty

        # Create the proposed dataframe
        self.proposed_df = pd.concat([self._default_df, activated_df], ignore_index=True)

        # Compute the proclivity for each proposition
        self.proposed_df['proclivity'] = self.proposed_df['weight'] * self.proposed_df['valence']

    def aggregate_propositions(self):
        """Aggregate the proclivity"""
        # Aggregate the proclivity for each decision
        grouped_df = self.proposed_df.groupby('decision').agg({'proclivity': 'sum', 'actions': 'first', # 'action': 'first', 
                                                               'length': 'first', 'intention': 'first', 'primitive': 'first'}).reset_index()
        # For each proposed composite decision 
        for index, proposed in grouped_df[grouped_df['length'] > 1].iterrows():
            # print(f"Index {index}, actions {proposition['actions']}, intention {proposition['intention']}")
            # Find shorter decisions that start with the same sequence 
            for _, shorter in self.proposed_df[self.proposed_df.apply(lambda row: proposed['actions'].startswith(row['actions']) 
                                                                      and row['length'] < proposed['length'], axis=1)].iterrows():
                # Add the proclivity of the shorter decisions
                grouped_df.loc[index, 'proclivity'] += shorter['proclivity']
                # print(f"Decision {proposed['decision']} recieves {shorter['proclivity']} from partial {shorter['intention']}")
        
        # Sort by descending proclivity
        self.decision_df = grouped_df.sort_values(by=['proclivity', 'decision'], ascending=[False, True]).reset_index(drop=True)

    def decide(self):
        """Selects the intended_interaction that at the top of the proposed dataframe"""
        # The intended interaction is in the first row because it has been sorted by descending proclivity
        intended_interaction_key = self.decision_df.loc[0, 'intention']
        print("Intention:", intended_interaction_key)
        self._intended_interaction = self._interactions[intended_interaction_key]

# Implémentons l'environnement SmallLoop 

In [7573]:
save_dir = "sav"

FEEL_FRONT = 0
FORWARD = 1
FEEL_LEFT = 2
FEEL_RIGHT = 3
TURN_LEFT = 4
TURN_RIGHT = 5


Créons l'environnement Small Loop

In [7574]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, BoundaryNorm
from ipywidgets import Button, HBox,VBox, Output
from IPython.display import display

LEFT = 0
DOWN = 1
RIGHT = 2
UP = 3
FEELING_EMPTY = 2
FEELING_WALL = 3
BUMPING = 4

colors = ["#b0b0b0", '#b0b0b0', '#ffffff', '#535865', "#F93943"]
agent_color = "#1976D2"

class SmallLoop():
    def __init__(self, position, direction):
        self.grid = np.array([
            [1, 1, 1, 1, 1, 1], 
            [1, 0, 0, 0, 1, 1],
            [1, 0, 1, 0, 0, 1],
            [1, 0, 1, 1, 0, 1],
            [1, 0, 0, 0, 0, 1],
            [1, 1, 1, 1, 1, 1]
        ])
        self.maze = self.grid.copy()
        self.position = np.array(position)  # Using NumPy array of shape (2)
        self.direction = direction
        self.cmap = ListedColormap(colors)
        self.norm = BoundaryNorm([-0.5, 0.5, 1.5, 2.5, 3.5, 4.5], self.cmap.N)
        self.marker_size = 400
        self.marker_map = {LEFT: '<', DOWN: 'v', RIGHT: '>', UP: '^'}
        self.marker_color = agent_color
        self.directions = np.array([
            [0, -1],  # Left
            [1, 0],   # Down
            [0, 1],   # Right
            [-1, 0]   # Up
            ])

    def outcome(self, action):
        """Update the grid. Return the outcome of the action."""
        result = 0
        # x, y = self.position

        if action == FORWARD:  
            target_position = self.position + self.directions[self.direction]
            if self.grid[tuple(target_position)] == 0:
                self.position[:] = target_position
            else:
                result = 1
                self.maze[tuple(target_position)] = BUMPING
        
        elif action == TURN_RIGHT:
            self.direction = {LEFT: UP, DOWN: LEFT, RIGHT: DOWN, UP: RIGHT}[self.direction]
        
        elif action == TURN_LEFT:
            self.direction = {LEFT: DOWN, DOWN: RIGHT, RIGHT: UP, UP: LEFT}[self.direction]
        
        elif action == FEEL_FRONT:
            feeling_position = self.position + self.directions[self.direction]
            if self.grid[tuple(feeling_position)] == 0:
                self.maze[tuple(feeling_position)] = FEELING_EMPTY
            else:
                result = 1
                self.maze[tuple(feeling_position)] = FEELING_WALL
        
        elif action == FEEL_LEFT:
            feeling_position = self.position + self.directions[(self.direction + 1) % 4]
            if self.grid[tuple(feeling_position)] == 0:
                self.maze[tuple(feeling_position)] = FEELING_EMPTY
            else:
                result = 1
                self.maze[tuple(feeling_position)] = FEELING_WALL
        
        elif action == FEEL_RIGHT:
            feeling_position = self.position + self.directions[self.direction - 1]
            if self.grid[tuple(feeling_position)] == 0:
                self.maze[tuple(feeling_position)] = FEELING_EMPTY
            else:
                result = 1
                self.maze[tuple(feeling_position)] = FEELING_WALL

        print(f"Line: {self.position[0]}, Column: {self.position[1]}, direction: {self.direction}")
        return result  
    
    def display(self):
        """Display the grid in the notebook"""
        out.clear_output(wait=True)
        with out:
            fig, ax = plt.subplots()
            ax.imshow(self.maze, cmap=self.cmap, norm=self.norm)
            plt.scatter(self.position[1], self.position[0], s=self.marker_size, marker=self.marker_map[self.direction], c=self.marker_color)
            ax.text(4.5, 0, f"{step:>3}", fontsize=12, color='White')
            plt.show()
    
    def save(self, step):
        """Save the display as a PNG file"""
        fig, ax = plt.subplots()
        ax.set_xticks([])
        ax.set_yticks([])
        ax.axis('off')
        ax.imshow(self.maze, cmap=self.cmap, norm=self.norm)
        plt.scatter(self.position[1], self.position[0], s=self.marker_size, marker=self.marker_map[self.direction], c=self.marker_color)
        ax.text(4.5, 0, f"{step:>4}", fontsize=12, color='White')
        plt.savefig(f"{save_dir}/{step:04}.png", bbox_inches='tight', pad_inches=0, transparent=True)
        plt.close(fig)
    
    def clear(self, clear):
        """Clear the grid display"""
        if clear:
            self.maze[:, :] = self.grid


# Testons l'agent dans le Small Loop

In [7575]:
# Instanciate the small loop environment
e = SmallLoop([1, 1], 0)

# Instanciate the agent 
interactions = [
    Interaction(FORWARD,0,5),
    Interaction(FORWARD,1,-10),
    Interaction(TURN_LEFT,0,-3),
    Interaction(TURN_LEFT,1,-3),
    Interaction(TURN_RIGHT,0,-3),
    Interaction(TURN_RIGHT,1,-3),
    Interaction(FEEL_FRONT,0,-1),
    Interaction(FEEL_FRONT,1,-1),
    Interaction(FEEL_LEFT,0,-1),
    Interaction(FEEL_LEFT,1,-1),
    Interaction(FEEL_RIGHT,0,-1),
    Interaction(FEEL_RIGHT,1,-1)
]
a = Agent(interactions)

# Initialize the interaction loop
step = 0
outcome = 0

# Display
out = Output()
e.display()
display(out)

Output()

In [7576]:
print(f"Step: {step}")
step += 1
action = a.action(outcome)
e.display()
e.save(step)  # Save the image file 
e.clear(a.clear)  # Clear the display grid if the current composite enaction is over
outcome = e.outcome(action)
a.proposed_df

Step: 0
Action: 0, Prediction: 0, Outcome: 0, Prediction_correct: True, Valence: -1
Intention: 00
Line: 1, Column: 1, direction: 0


,activated,weight,actions,intention,valence,decision,length,primitive,proclivity
0,,0,1,10,5,1,1,10,0
1,,0,4,40,-3,4,1,40,0
2,,0,5,50,-3,5,1,50,0
3,,0,0,00,-1,0,1,00,0
4,,0,2,20,-1,2,1,20,0
5,,0,3,30,-1,3,1,30,0


Nous observons que l'agent construit rapidement des interactions composites très longues qui ont une valence élevée car elles contiennent plusieurs interactions avancer, mais qui échouent la plupart du temps. 

Attendre que les interactions composites aient fait leur preuve en étant énactée entièrement avant de baser des interactions de plus haut niveau sur elles.


# Implémentons l'Agent11

Modifions la méthode `create_proposed_df()` pour proposer des interactions uniquement si leur poinds est suppérieur à `confidence_threshold`.

In [7577]:
confidence_threshold = 1

In [7578]:
class Agent11(Agent):
    def create_proposed_df(self):
        """Create the proposed dataframe from the activated interactions"""
        # The list of activated interaction that match the current context
        activated_keys = [composite_interaction.key() for composite_interaction in self._interactions.values()
                          if composite_interaction.get_length() > 1 
                          and (composite_interaction.pre_interaction == self._last_interaction 
                               or composite_interaction.pre_interaction == self._last_composite_interaction
                               or (self._last_interaction.get_length() > 1
                                  and composite_interaction.pre_interaction == self._last_interaction.post_interaction))]
        data = {'activated': activated_keys,
                'weight': [self._interactions[k].weight for k in activated_keys],
                # 'action': [self._interactions[k].post_interaction.get_primitive_action() for k in activated_keys],
                'actions': [self._interactions[k].post_interaction.get_actions()  if self._interactions[k].post_interaction.weight > confidence_threshold else self._interactions[k].post_interaction.pre_interaction.get_actions() for k in activated_keys],
                'intention': [self._interactions[k].post_interaction.key() if self._interactions[k].post_interaction.weight > confidence_threshold else self._interactions[k].post_interaction.pre_interaction.key() for k in activated_keys],
                'valence': [self._interactions[k].post_interaction.get_valence() for k in activated_keys],
                'decision': [self._interactions[k].post_interaction.get_decision() if self._interactions[k].post_interaction.weight > confidence_threshold else self._interactions[k].post_interaction.pre_interaction.get_decision() for k in activated_keys],
                'primitive': [self._interactions[k].post_interaction.pre_key() for k in activated_keys],
                'length': [self._interactions[k].post_interaction.get_length() for k in activated_keys],
                }
        activated_df = pd.DataFrame(data).astype(self._default_df.dtypes)  # Force the same types for the case it is empty

        # Create the proposed dataframe
        self.proposed_df = pd.concat([self._default_df, activated_df], ignore_index=True).sort_values(by='decision', ascending=True).reset_index(drop=True)

        # Compute the proclivity for each proposition
        self.proposed_df['proclivity'] = self.proposed_df['weight'] * self.proposed_df['valence']


# Testons l'Agent11 dans le small loop

In [7579]:
# Instanciate the small loop environment
e = SmallLoop([1, 1], 0)

# Instanciate the agent 
interactions = [
    Interaction(FORWARD,0,5),
    Interaction(FORWARD,1,-10),
    Interaction(TURN_LEFT,0,-2),
    Interaction(TURN_LEFT,1,-2),
    Interaction(TURN_RIGHT,0,-5),
    Interaction(TURN_RIGHT,1,-2),
    Interaction(FEEL_FRONT,0,-1),
    Interaction(FEEL_FRONT,1,-1),
    Interaction(FEEL_LEFT,0,-1),
    Interaction(FEEL_LEFT,1,-1),
    Interaction(FEEL_RIGHT,0,-1),
    Interaction(FEEL_RIGHT,1,-1)
]
a = Agent11(interactions)

# Initialize the interaction loop
step = 0
outcome = 0

# Display
out = Output()
e.display()
display(out)

Output()

In [7580]:
print(f"Step: {step}")
step += 1
action = a.action(outcome)
e.display()
e.save(step)  # Save the image file 
e.clear(a.clear)  # Clear the display grid if the current composite enaction is over
outcome = e.outcome(action)
a.proposed_df

Step: 0
Action: 0, Prediction: 0, Outcome: 0, Prediction_correct: True, Valence: -1
Intention: 00
Line: 1, Column: 1, direction: 0


,activated,weight,actions,intention,valence,decision,length,primitive,proclivity
0,,0,0,00,-1,0,1,00,0
1,,0,1,10,5,1,1,10,0
2,,0,2,20,-1,2,1,20,0
3,,0,3,30,-1,3,1,30,0
4,,0,4,40,-2,4,1,40,0
5,,0,5,50,-5,5,1,50,0


# 1000 cycles d'interaction

In [7581]:
for step in range(1200):
    print(f"Step: {step}")
    # step += 1
    action = a.action(outcome)
    e.display()
    e.save(step)  # Save the image file 
    e.clear(a.clear)  # Clear the display grid if the current composite enaction is over
    outcome = e.outcome(action)
# a.proposed_df

Step: 0
Action: 0, Prediction: 0, Outcome: 1, Prediction_correct: False, Valence: -1
Learning (00:-1, 01:-1: 1)
Intention: 00
Line: 1, Column: 1, direction: 0
Step: 1
Action: 0, Prediction: 0, Outcome: 1, Prediction_correct: False, Valence: -1
Learning (01:-1, 01:-1: 1)
Learning ((00:-1, 01:-1: 1), 01:-1: 1)
Learning (00:-1, (01:-1, 01:-1: 1): 1)
Intention: 10
Line: 1, Column: 1, direction: 0
Step: 2
Action: 1, Prediction: 0, Outcome: 1, Prediction_correct: False, Valence: -10
Learning (01:-1, 11:-10: 1)
Learning ((01:-1, 01:-1: 1), 11:-10: 1)
Learning (01:-1, (01:-1, 11:-10: 1): 1)
Learning ((00:-1, 01:-1: 1), (01:-1, 11:-10: 1): 1)
Intention: 00
Line: 1, Column: 1, direction: 0
Step: 3
Action: 0, Prediction: 0, Outcome: 1, Prediction_correct: False, Valence: -1
Learning (11:-10, 01:-1: 1)
Learning ((01:-1, 11:-10: 1), 01:-1: 1)
Learning (01:-1, (11:-10, 01:-1: 1): 1)
Learning ((01:-1, 01:-1: 1), (11:-10, 01:-1: 1): 1)
Intention: 20
Line: 1, Column: 1, direction: 0
Step: 4
Action: 2, 

Exception ignored in: <function WeakMethod.__new__.<locals>._cb at 0x00000260FE246550>
Traceback (most recent call last):
  File "c:\users\ogeorgeon\appdata\local\programs\python\python39\lib\weakref.py", line 61, in _cb
    callback(self)
  File "c:\users\ogeorgeon\appdata\local\programs\python\python39\lib\site-packages\matplotlib\cbook.py", line 248, in _remove_proxy
    del self.callbacks[signal][cid]
KeyError: 'changed'


Line: 2, Column: 3, direction: 0
Step: 69
Action: 0, Prediction: 0, Outcome: 1, Prediction_correct: False, Valence: -1
Learning (((00:-1, 10:5: 2), 01:-1: 4), 01:-1: 1)
Learning ((40:-2, ((00:-1, 10:5: 2), 01:-1: 4): 2), 01:-1: 1)
Learning (40:-2, (((00:-1, 10:5: 2), 01:-1: 4), 01:-1: 1): 1)
Learning ((01:-1, 40:-2: 4), (((00:-1, 10:5: 2), 01:-1: 4), 01:-1: 1): 1)
Intention: 40
Line: 2, Column: 3, direction: 1
Step: 70
Action: 4, Prediction: 0, Outcome: 0, Prediction_correct: True, Valence: -2
Reinforcing (01:-1, 40:-2: 5)
Learning ((((00:-1, 10:5: 2), 01:-1: 4), 01:-1: 1), 40:-2: 1)
Learning (((00:-1, 10:5: 2), 01:-1: 4), (01:-1, 40:-2: 5): 1)
Learning ((40:-2, ((00:-1, 10:5: 2), 01:-1: 4): 2), (01:-1, 40:-2: 5): 1)
Intention: (((00,10),(00,10)),01)
Line: 2, Column: 3, direction: 1
Step: 71
Action: 0, Prediction: 0, Outcome: 1, Prediction_correct: False, Valence: -1
Reinforcing (40:-2, 01:-1: 5)
Reinforcing ((01:-1, 40:-2: 5), 01:-1: 2)
Reinforcing (01:-1, (40:-2, 01:-1: 5): 2)
Learni

Exception ignored in: <function WeakMethod.__new__.<locals>._cb at 0x00000260F9CC8C10>
Traceback (most recent call last):
  File "c:\users\ogeorgeon\appdata\local\programs\python\python39\lib\weakref.py", line 61, in _cb
    callback(self)
  File "c:\users\ogeorgeon\appdata\local\programs\python\python39\lib\site-packages\matplotlib\cbook.py", line 248, in _remove_proxy
    del self.callbacks[signal][cid]
KeyError: 'changed'


Line: 1, Column: 2, direction: 0
Step: 335
Action: 0, Prediction: 0, Outcome: 0, Prediction_correct: True, Valence: -1
Line: 1, Column: 1, direction: 0
Step: 336
Action: 1, Prediction: 0, Outcome: 0, Prediction_correct: True, Valence: 5
Line: 1, Column: 1, direction: 0
Step: 337
Action: 0, Prediction: 1, Outcome: 1, Prediction_correct: True, Valence: -1
Line: 1, Column: 1, direction: 0
Step: 338
Action: 2, Prediction: 0, Outcome: 0, Prediction_correct: True, Valence: -1
Line: 1, Column: 1, direction: 1
Step: 339
Action: 4, Prediction: 0, Outcome: 0, Prediction_correct: True, Valence: -2
Line: 1, Column: 1, direction: 1
Step: 340
Action: 0, Prediction: 0, Outcome: 0, Prediction_correct: True, Valence: -1
Line: 2, Column: 1, direction: 1
Step: 341
Action: 1, Prediction: 0, Outcome: 0, Prediction_correct: True, Valence: 5
Reinforcing (20:-1, (((40:-2, (00:-1, 10:5: 2): 2), ((00:-1, 10:5: 2), 01:-1: 16): 2), (20:-1, (40:-2, (00:-1, 10:5: 2): 2): 3): 2): 2)
Learning (((10:5, 01:-1: 4), 20:-

Exception ignored in: <function WeakMethod.__new__.<locals>._cb at 0x00000260F9AE99D0>
Traceback (most recent call last):
  File "c:\users\ogeorgeon\appdata\local\programs\python\python39\lib\weakref.py", line 61, in _cb
    callback(self)
  File "c:\users\ogeorgeon\appdata\local\programs\python\python39\lib\site-packages\matplotlib\cbook.py", line 248, in _remove_proxy
    del self.callbacks[signal][cid]
KeyError: 'changed'


Line: 2, Column: 3, direction: 1
Step: 566
Action: 0, Prediction: 0, Outcome: 1, Prediction_correct: False, Valence: -1
Reinforcing ((30:-1, 40:-2: 3), 01:-1: 4)
Learning ((01:-1, (30:-1, 40:-2: 3): 2), 01:-1: 1)
Learning (01:-1, ((30:-1, 40:-2: 3), 01:-1: 4): 1)
Learning ((21:-1, 01:-1: 8), ((30:-1, 40:-2: 3), 01:-1: 4): 1)
Intention: (30,40)
Line: 2, Column: 3, direction: 1
Step: 567
Action: 3, Prediction: 0, Outcome: 1, Prediction_correct: False, Valence: -1
Reinforcing (01:-1, 31:-1: 8)
Learning (((30:-1, 40:-2: 3), 01:-1: 4), 31:-1: 1)
Reinforcing ((30:-1, 40:-2: 3), (01:-1, 31:-1: 8): 3)
Learning ((01:-1, (30:-1, 40:-2: 3): 2), (01:-1, 31:-1: 8): 1)
Intention: (40,((00,10),(00,10)))
Line: 2, Column: 3, direction: 2
Step: 568
Action: 4, Prediction: 0, Outcome: 0, Prediction_correct: True, Valence: -2
Line: 2, Column: 3, direction: 2
Step: 569
Action: 0, Prediction: 0, Outcome: 0, Prediction_correct: True, Valence: -1
Line: 2, Column: 4, direction: 2
Step: 570
Action: 1, Prediction

Exception ignored in: <function WeakMethod.__new__.<locals>._cb at 0x00000260FE0FB1F0>
Traceback (most recent call last):
  File "c:\users\ogeorgeon\appdata\local\programs\python\python39\lib\weakref.py", line 61, in _cb
    callback(self)
  File "c:\users\ogeorgeon\appdata\local\programs\python\python39\lib\site-packages\matplotlib\cbook.py", line 248, in _remove_proxy
    del self.callbacks[signal][cid]
KeyError: 'changed'


Line: 2, Column: 3, direction: 0
Step: 740
Action: 1, Prediction: 0, Outcome: 1, Prediction_correct: False, Valence: -10
Reinforcing (21:-1, 11:-10: 7)
Learning ((11:-10, 21:-1: 3), 11:-10: 1)
Learning (11:-10, (21:-1, 11:-10: 7): 1)
Learning ((30:-1, 11:-10: 2), (21:-1, 11:-10: 7): 1)
Intention: 50
Line: 2, Column: 3, direction: 3
Step: 741
Action: 5, Prediction: 0, Outcome: 0, Prediction_correct: True, Valence: -5
Reinforcing (11:-10, 50:-5: 2)
Learning ((21:-1, 11:-10: 7), 50:-5: 1)
Learning (21:-1, (11:-10, 50:-5: 2): 1)
Learning ((11:-10, 21:-1: 3), (11:-10, 50:-5: 2): 1)
Intention: ((21,(10,((00,10),00))),00)
Line: 2, Column: 3, direction: 3
Step: 742
Action: 2, Prediction: 1, Outcome: 1, Prediction_correct: True, Valence: -1
Line: 1, Column: 3, direction: 3
Step: 743
Action: 1, Prediction: 0, Outcome: 0, Prediction_correct: True, Valence: 5
Line: 1, Column: 3, direction: 3
Step: 744
Action: 0, Prediction: 0, Outcome: 1, Prediction_correct: False, Valence: -1
Reinforcing (10:5, 0

# Créons le film gif

In [7570]:
import imageio.v2 as imageio
import os

img_dir = f"./{save_dir}"
all_files = [os.path.join(img_dir, f) for f in os.listdir(img_dir) if f.endswith('.png')]
images = [imageio.imread(f) for f in all_files]
imageio.mimsave("movie.gif", images, fps=5)